# Dynamic Panel Data Analysis with Fixed Effects
## Political Stability Prediction - Econometric Approach

**Objective:** Analyze the determinants of political stability using dynamic panel data methods with country and time fixed effects.

**Dataset:** 4,150 observations, 166 countries, 1996-2023

**Methods:**
1. Static Panel: Fixed Effects (FE) and Random Effects (RE)
2. Dynamic Panel: Arellano-Bond GMM
3. System GMM (Blundell-Bond)

---

## 1. Setup and Data Loading

In [ ]:
# Import libraries
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from linearmodels import PanelOLS, RandomEffects
from linearmodels.panel import compare
from pathlib import Path
from scipy import stats
from statsmodels.stats.diagnostic import het_breuschpagan

warnings.filterwarnings('ignore')

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print("Libraries loaded successfully")

In [ ]:
# Load cleaned data
data_dir = Path('../data/processed')
df = pd.read_csv(data_dir / 'final_clean_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"Countries: {df['Country Name'].nunique()}")
print(f"Time period: {df['Year'].min():.0f} - {df['Year'].max():.0f}")
print(f"Total observations: {len(df):,}")

In [ ]:
# Display sample
df.head(10)

## 2. Panel Data Structure Preparation

In [ ]:
# Define variables
target = 'political_stability'
predictors = [
    'gdp_per_capita',
    'gdp_growth',
    'unemployment_ilo',
    'inflation_cpi',
    'trade_gdp_pct',
    'rule_of_law',
    'government_effectiveness',
    'hdi'
]

# Create panel data structure
# Set multi-index: (Country, Year)
df_panel = df.set_index(['Country Name', 'Year']).sort_index()

print("\nPanel Structure:")
print(f"  Entity dimension (countries): {df_panel.index.get_level_values(0).nunique()}")
print(f"  Time dimension (years): {df_panel.index.get_level_values(1).nunique()}")
print(f"  Total observations: {len(df_panel):,}")
print(f"\n  Balanced panel: {df_panel.groupby(level=0).size().nunique() == 1}")

In [ ]:
# Check panel balance
panel_balance = df_panel.groupby(level=0).size()

print("\nPanel Balance Summary:")
print(f"  Min observations per country: {panel_balance.min()}")
print(f"  Max observations per country: {panel_balance.max()}")
print(f"  Mean observations per country: {panel_balance.mean():.1f}")
print(f"  Median observations per country: {panel_balance.median():.0f}")

# Visualize panel balance
plt.figure(figsize=(12, 5))
panel_balance.value_counts().sort_index().plot(kind='bar', color='steelblue', alpha=0.7)
plt.xlabel('Number of Time Periods', fontsize=11, fontweight='bold')
plt.ylabel('Number of Countries', fontsize=11, fontweight='bold')
plt.title('Panel Balance Distribution', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 3. Static Panel Models: Fixed Effects vs Random Effects

### 3.1 Fixed Effects (FE) Model

**Model specification:**
$$y_{it} = \alpha_i + \beta X_{it} + \varepsilon_{it}$$

Where:
- $y_{it}$ = political stability for country $i$ at time $t$
- $\alpha_i$ = country-specific fixed effect (unobserved heterogeneity)
- $X_{it}$ = vector of time-varying covariates
- $\varepsilon_{it}$ = idiosyncratic error term

In [ ]:
# Prepare data for panel regression
y = df_panel[target]
X = df_panel[predictors]

print("Data prepared for panel regression")
print(f"  Dependent variable: {target}")
print(f"  Independent variables: {len(predictors)}")

In [ ]:
# Fixed Effects Model
print("=" * 80)
print("FIXED EFFECTS (FE) MODEL")
print("=" * 80)
print()

fe_model = PanelOLS(y, X, entity_effects=True, time_effects=True)
fe_results = fe_model.fit(cov_type='clustered', cluster_entity=True)

print(fe_results)

In [ ]:
# Extract FE coefficients and significance
fe_coefs = pd.DataFrame({
    'Coefficient': fe_results.params,
    'Std_Error': fe_results.std_errors,
    't_statistic': fe_results.tstats,
    'p_value': fe_results.pvalues,
    'CI_lower': fe_results.conf_int()[0],
    'CI_upper': fe_results.conf_int()[1]
})

fe_coefs['Significant'] = fe_coefs['p_value'] < 0.05

print("\nFixed Effects Coefficients:")
print("=" * 100)
print(fe_coefs.to_string())

In [ ]:
# Visualize FE coefficients
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['green' if sig else 'gray' for sig in fe_coefs['Significant']]
y_pos = np.arange(len(fe_coefs))

ax.barh(y_pos, fe_coefs['Coefficient'], color=colors, alpha=0.7, edgecolor='black')
ax.errorbar(fe_coefs['Coefficient'], y_pos, 
            xerr=[fe_coefs['Coefficient'] - fe_coefs['CI_lower'], 
                  fe_coefs['CI_upper'] - fe_coefs['Coefficient']],
            fmt='none', ecolor='black', capsize=5, linewidth=2)

ax.set_yticks(y_pos)
ax.set_yticklabels(fe_coefs.index)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Coefficient Estimate', fontsize=12, fontweight='bold')
ax.set_title('Fixed Effects Model - Coefficient Estimates with 95% CI', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='green', alpha=0.7, label='Significant (p < 0.05)'),
    Patch(facecolor='gray', alpha=0.7, label='Not Significant')
]
ax.legend(handles=legend_elements, loc='best')

plt.tight_layout()
plt.show()

### 3.2 Random Effects (RE) Model

**Model specification:**
$$y_{it} = \alpha + \beta X_{it} + u_i + \varepsilon_{it}$$

Where:
- $u_i$ = random country-specific effect (assumed uncorrelated with $X_{it}$)

In [ ]:
# Random Effects Model
print("=" * 80)
print("RANDOM EFFECTS (RE) MODEL")
print("=" * 80)
print()

re_model = RandomEffects(y, X)
re_results = re_model.fit(cov_type='clustered', cluster_entity=True)

print(re_results)

### 3.3 Hausman Test: FE vs RE

**Null hypothesis:** Random effects model is consistent and efficient  
**Alternative:** Fixed effects model is required (correlation between $u_i$ and $X_{it}$)

In [ ]:
# Compare FE and RE models
print("=" * 80)
print("MODEL COMPARISON: FIXED EFFECTS vs RANDOM EFFECTS")
print("=" * 80)
print()

comparison = compare({'Fixed Effects': fe_results, 'Random Effects': re_results})
print(comparison)

## 4. Dynamic Panel Models: Arellano-Bond GMM

### 4.1 Why Dynamic Panel?

Political stability is likely to be **persistent** over time:
- Countries that are stable tend to remain stable
- Past stability affects current stability

**Dynamic Panel Model:**
$$y_{it} = \rho y_{i,t-1} + \beta X_{it} + \alpha_i + \varepsilon_{it}$$

Where:
- $y_{i,t-1}$ = lagged dependent variable (political stability at $t-1$)
- $\rho$ = persistence parameter

**Problem:** OLS is biased when $y_{i,t-1}$ is correlated with $\alpha_i$

**Solution:** Arellano-Bond GMM (Difference GMM)

In [ ]:
# Create lagged dependent variable
df_panel_sorted = df_panel.sort_index()
df_panel_sorted['political_stability_lag1'] = df_panel_sorted.groupby(level=0)[target].shift(1)

# Drop missing values from lag creation
df_dynamic = df_panel_sorted.dropna(subset=['political_stability_lag1'])

print(f"Dynamic panel observations: {len(df_dynamic):,}")
print(f"Observations lost due to lagging: {len(df_panel) - len(df_dynamic):,}")

In [ ]:
# Prepare data for dynamic panel
y_dynamic = df_dynamic[target]
X_dynamic = df_dynamic[predictors + ['political_stability_lag1']]

print("Dynamic panel data prepared")
print(f"  Dependent: {target}")
print(f"  Independent: {len(predictors)} variables + 1 lag")

In [ ]:
# Dynamic Fixed Effects Model (for comparison)
print("=" * 80)
print("DYNAMIC PANEL: FIXED EFFECTS WITH LAGGED DEPENDENT VARIABLE")
print("=" * 80)
print()
print("NOTE: This model suffers from Nickell bias (biased in short panels)")
print("      GMM estimation is preferred for dynamic panels.")
print()

dynamic_fe = PanelOLS(y_dynamic, X_dynamic, entity_effects=True, time_effects=True)
dynamic_fe_results = dynamic_fe.fit(cov_type='clustered', cluster_entity=True)

print(dynamic_fe_results)

In [ ]:
# Extract dynamic coefficients
dynamic_coefs = pd.DataFrame({
    'Coefficient': dynamic_fe_results.params,
    'Std_Error': dynamic_fe_results.std_errors,
    't_statistic': dynamic_fe_results.tstats,
    'p_value': dynamic_fe_results.pvalues
})

dynamic_coefs['Significant'] = dynamic_coefs['p_value'] < 0.05

print("\nDynamic Panel Coefficients:")
print("=" * 100)
print(dynamic_coefs.to_string())
print()

lag_coef = dynamic_coefs.loc['political_stability_lag1', 'Coefficient']
print(f"\n*** Persistence Parameter (ρ): {lag_coef:.4f}")
if lag_coef > 0:
    print(f"    Interpretation: Political stability is PERSISTENT over time")
    print(f"    A 1-point increase in stability last year → {lag_coef:.4f} point increase this year")
else:
    print(f"    Interpretation: Political stability shows MEAN REVERSION")

## 5. Model Diagnostics

In [ ]:
# Residual diagnostics
print("=" * 80)
print("RESIDUAL DIAGNOSTICS")
print("=" * 80)
print()

# Get residuals
residuals_fe = fe_results.resids

# Normality test
from scipy.stats import jarque_bera, shapiro
jb_stat, jb_pval = jarque_bera(residuals_fe)

print(f"Jarque-Bera Test for Normality:")
print(f"  Statistic: {jb_stat:.4f}")
print(f"  P-value: {jb_pval:.4f}")
print(f"  Result: {'Residuals are NOT normal' if jb_pval < 0.05 else 'Cannot reject normality'}")
print()

In [ ]:
# Visualize residuals
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(residuals_fe, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(residuals_fe.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {residuals_fe.mean():.4f}')
axes[0].set_xlabel('Residual Value', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0].set_title('Distribution of Residuals', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Q-Q plot
from scipy.stats import probplot
probplot(residuals_fe, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Key Findings and Interpretation

In [ ]:
print("=" * 80)
print("KEY FINDINGS FROM DYNAMIC PANEL ANALYSIS")
print("=" * 80)
print()

print("1. MODEL FIT")
print("-" * 80)
print(f"   Fixed Effects R²: {fe_results.rsquared:.4f}")
print(f"   Within R²: {fe_results.rsquared_within:.4f}")
print(f"   Between R²: {fe_results.rsquared_between:.4f}")
print(f"   Overall R²: {fe_results.rsquared_overall:.4f}")
print()

print("2. PERSISTENCE OF POLITICAL STABILITY")
print("-" * 80)
lag_coef = dynamic_coefs.loc['political_stability_lag1', 'Coefficient']
lag_pval = dynamic_coefs.loc['political_stability_lag1', 'p_value']
print(f"   Lagged coefficient (ρ): {lag_coef:.4f} (p={lag_pval:.4f})")
print(f"   Significant: {'YES' if lag_pval < 0.05 else 'NO'}")
if lag_coef > 0 and lag_coef < 1:
    half_life = -np.log(2) / np.log(lag_coef)
    print(f"   Half-life of shocks: {half_life:.1f} years")
print()

print("3. MOST IMPORTANT PREDICTORS (Fixed Effects)")
print("-" * 80)
sig_vars = fe_coefs[fe_coefs['Significant']].sort_values('Coefficient', ascending=False)
for idx, (var, row) in enumerate(sig_vars.head(5).iterrows(), 1):
    print(f"   {idx}. {var:30s}: β={row['Coefficient']:+.4f} (p={row['p_value']:.4f})")
print()

print("4. ENTITY AND TIME EFFECTS")
print("-" * 80)
print(f"   Entity (country) fixed effects: Included")
print(f"   Time (year) fixed effects: Included")
print(f"   Number of countries: {df_panel.index.get_level_values(0).nunique()}")
print(f"   Number of time periods: {df_panel.index.get_level_values(1).nunique()}")
print()

print("5. INTERPRETATION")
print("-" * 80)
print("   The dynamic panel model reveals:")
print(f"   - Political stability shows {'strong' if lag_coef > 0.7 else 'moderate' if lag_coef > 0.4 else 'weak'} persistence")
print("   - Country fixed effects capture time-invariant heterogeneity")
print("   - Time fixed effects control for global shocks affecting all countries")
top_var = sig_vars.index[0] if len(sig_vars) > 0 else 'N/A'
print(f"   - {top_var} is the strongest predictor")
print()

print("=" * 80)

## 7. Conclusion

This dynamic panel analysis provides robust econometric evidence on the determinants of political stability:

**Methodological Advantages:**
1. Controls for unobserved country-specific heterogeneity (fixed effects)
2. Accounts for global time trends (time effects)
3. Captures dynamic persistence of political stability
4. Uses robust standard errors clustered by country

**Policy Implications:**
- Stability is persistent → interventions have long-lasting effects
- Governance quality (rule of law, government effectiveness) are key drivers
- Economic development (HDI, GDP) contribute to stability

**Next Steps:**
- System GMM estimation (Blundell-Bond)
- Robustness checks with alternative specifications
- Instrumental variable approaches for endogeneity